In [1]:
import logging
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

C:\Users\moon\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [11]:
from tqdm import tqdm

In [3]:
import gensim
gensim.__version__

'2.1.0'

In [4]:
import os 
os.getcwd()

'C:\\Users\\moon\\Google 드라이브\\경배끄10\\한이음2(추천시스템)\\Week4'

In [5]:
os.chdir('C:\\Users\\Moon\\Google 드라이브\\경배끄10\\한이음2(추천시스템)\\fc-recsys-lecture-master\\fc-recsys-lecture-master\\week4\\resource\\mr.150813')

In [6]:
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: ECD2-C70F

 C:\Users\Moon\Google 드라이브\경배끄10\한이음2(추천시스템)\fc-recsys-lecture-master\fc-recsys-lecture-master\week4\resource\mr.150813 디렉터리

2017-06-09  오후 08:14    <DIR>          .
2017-06-09  오후 08:14    <DIR>          ..
2015-10-25  오후 01:08       120,409,230 mr.150813
               1개 파일         120,409,230 바이트
               2개 디렉터리  113,484,324,864 바이트 남음


In [7]:
review_file = 'mr.150813'
review_ds = pd.read_csv(review_file, sep='\t', index_col=None, header=None, names=['idx','title','review','rating','source','timestamp'])

C:\Users\moon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
print(review_ds.shape)
review_ds.head(3)

(697517, 6)


,idx,title,review,rating,source,timestamp
0,1,10억,몇년동안 본거중에 가장 최악이다 한국영화여서 챙피하다...,1,Naver movie,1437446025
1,2,10억,와. .진심 그 피디 죽일수있었는데 못죽이고ㅡㅡ해튼 주연이라는 신민아 연기는 아주 ...,1,Naver movie,1437446025
2,3,12 몽키즈,킬링타임 용도로도 써먹지 못할 수준,1,Naver movie,1437446025


In [9]:
from datetime import datetime

review_ds['timestamp'] = datetime.strptime('3/16/2016 10:38', '%m/%d/%Y %H:%M')
print(review_ds.shape)
review_ds.head(3)

(697517, 6)


,idx,title,review,rating,source,timestamp
0,1,10억,몇년동안 본거중에 가장 최악이다 한국영화여서 챙피하다...,1,Naver movie,2016-03-16 10:38:00
1,2,10억,와. .진심 그 피디 죽일수있었는데 못죽이고ㅡㅡ해튼 주연이라는 신민아 연기는 아주 ...,1,Naver movie,2016-03-16 10:38:00
2,3,12 몽키즈,킬링타임 용도로도 써먹지 못할 수준,1,Naver movie,2016-03-16 10:38:00


## 영화 리뷰 데이터셋 트레이닝

In [12]:
# Nan value drop
review_ds = review_ds.dropna()

# corpus generation
sentences = []
for idx, row in tqdm(review_ds.iterrows()):
    words = str(row['review']).split()
    sentences.append(words)

697435it [01:07, 10342.20it/s]


In [14]:
sentences[:4]

[['몇년동안', '본거중에', '가장', '최악이다', '한국영화여서', '챙피하다...'],
 ['와.',
  '.진심',
  '그',
  '피디',
  '죽일수있었는데',
  '못죽이고ㅡㅡ해튼',
  '주연이라는',
  '신민아',
  '연기는',
  '아주',
  '발연기!!',
  '.',
  '.',
  '신민아',
  '와',
  '대박이다',
  '심하게',
  '못한다'],
 ['킬링타임', '용도로도', '써먹지', '못할', '수준'],
 ['영화는',
  '좋다만..',
  '실제',
  '주인공',
  '정신못차렸어..',
  '아내',
  '폭행죄로',
  '교도소행..',
  '감동이',
  '싹다',
  '사라졌다..']]

In [15]:
def myhashfxn(obj):
    return hash(obj)%(2**32)

In [19]:
model = Word2Vec(sentences, min_count =5, window=5, size=100, hashfxn=myhashfxn)

In [20]:
os.getcwd()

'C:\\Users\\Moon\\Google 드라이브\\경배끄10\\한이음2(추천시스템)\\fc-recsys-lecture-master\\fc-recsys-lecture-master\\week4\\resource\\mr.150813'

In [29]:
model.save('C:\\Users\\Moon\\Google 드라이브\\경배끄10\\한이음2(추천시스템)\\Week4\\movie_review.model')

In [30]:
model['이정재'][:3]

array([-0.22844557,  0.76812816, -0.35898805], dtype=float32)

In [31]:
## 인물 단어에 대한 유사도 
model.most_similar(positive=['이정재'], topn=3)

[('조진웅', 0.952551007270813),
 ('전지현', 0.9445124268531799),
 ('하정우', 0.9328281879425049)]

In [32]:
## 영화 단어에 대한 유사도
model.most_similar(positive=['암살'], topn=3)

[('명량', 0.899853527545929),
 ('군도', 0.8969316482543945),
 ('겨울왕국', 0.8670167326927185)]

In [33]:
model.most_similar(positive=['이정재','관상'],negative=['암살'], topn=3)

[('정재영,', 0.8134846687316895),
 ('김고은과', 0.8083153963088989),
 ('뿜어내는', 0.79880291223526)]

In [35]:
model.doesnt_match('이정재 하정우 전지현 백윤식'.split())

'백윤식'

## 단어 클러스터링 하기
$$k = sqrt(N/2)`s 올림 $$
* N = index의 길이

In [37]:
import math

k = math.ceil(math.sqrt(len(model.wv.index2word)/2))
print('cluster k의 수 {0}'.format(k))

cluster k의 수 239


In [42]:
vec = model.wv.syn0
index2word = model.wv.index2word
# index2word 는 리뷰에 대한 모든 words의 집합
# vec은 각 단어들에 대한 vector 값을 array로 만든것
print(vec.shape)

(113374, 100)


In [58]:
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=k)
%time cluster_model.fit(vec)

Wall time: 31min 20s


KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=239, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)